# Analysing NN dynamics using HSIC Criterion - Intermediate layers

In [1]:
# importing PyGlow modules
import glow
from glow.layers import Dense, Dropout, Conv2d, Flatten, HSICoutput
from glow.datasets import mnist, cifar10
from glow.models import IBSequential, Sequential, HSICSigma
from glow.information_bottelneck.estimator import HSIC

In [2]:
# hyperparameter
batch_size = 64
num_workers = 3
validation_split = 0.2
num_epochs = 2

# load the dataset
train_loader, val_loader, test_loader = mnist.load_data(
    batch_size=batch_size, num_workers=num_workers, validation_split=validation_split
)

In [3]:
# define the model
model = IBSequential(input_shape=(1, 28, 28), gpu=True, track_dynamics=True, save_dynamics=True)
model.add(Conv2d(filters=16, kernel_size=3, stride=1, padding=1))
model.add(Flatten())
model.add(Dropout(0.4))
model.add(Dense(500, activation='relu'))
model.add(Dropout(0.4))
model.add(Dense(200, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(10, activation='softmax'))
print(model)

Running on CUDA enabled device !
IBSequential(
  (layer_list): ModuleList(
    (0): Sequential(
      (0): Conv2d(
        (conv_layer): Conv2d(1, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      )
    )
    (1): Sequential(
      (0): Flatten()
    )
    (2): Sequential(
      (0): Dropout(
        (dropout_layer): Dropout(p=0.4)
      )
    )
    (3): Sequential(
      (0): Dense(
        (weights): Linear(in_features=12544, out_features=500, bias=True)
      )
    )
    (4): Sequential(
      (0): Dropout(
        (dropout_layer): Dropout(p=0.4)
      )
    )
    (5): Sequential(
      (0): Dense(
        (weights): Linear(in_features=500, out_features=200, bias=True)
      )
    )
    (6): Sequential(
      (0): Dropout(
        (dropout_layer): Dropout(p=0.2)
      )
    )
    (7): Sequential(
      (0): Dense(
        (weights): Linear(in_features=200, out_features=10, bias=True)
      )
    )
  )
)


## Attaching dynamics evaluator to the model
We can attach evaluators which calculates information plane coordinates for all layers for all batches for all epochs. Any custom evaluator can be passed as object in the .attach_evaluator.


In [4]:
# compile the model
model.compile(optimizer='SGD', loss='cross_entropy', metrics=['accuracy'])
model.attach_evaluator(HSIC(sigma=5, gpu=True))

In [ ]:
# train the model along with calculating dynamics
model.fit_generator(train_loader, val_loader, num_epochs)